<img src="img/bigsem.png" width="40%" align="right">
<img src="img/logo_wiwi.png" width="20%" align="left">





<br><br><br><br>

# Dynamic Programming Models in Combinatorial Optimization
**Winter Term 2024/25**


# 2a: Domain-Independent Dynamic Programming

<img src="img/decision_analytics_logo.png" width="17%" align="right">


<br>

<br>
<br>

**J-Prof. Dr. Michael Römer |  Decision Analytics Group**
                                                    


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numba import njit
from typing import NamedTuple, Callable
# import keyboard
from IPython.display import SVG, display, clear_output
import networkx as nx


# Overview
- Modelling Exercises
- Domain-Independent Dynamic Programming

## Example: the 0/1 knapsack problem

Given 
- a knapsack with a capacity $W$ 
- and a set of items, each with a weight $w_i$ and a value $p_i$
- determine the the subset of the items to put in the knapsack such that
  - the total value of the items in the knapsack is maximal and
  - the total weight of the items in the knapsack does not exceed $W$

## The Travelling Salesperson Problem

<img src="https://pup-assets.imgix.net/onix/images/9780691163529.jpg" width="20%" align="right">


**Informal problem statement:** Given a set of cities and the distances between the cities, find a minimum-cost round-trip that visits each city exactly once.

**More formally:** Given a complete graph and distances between each pair of nodes in the graph, find a cost-minimal hamiltonian cycle in the graph


- one of the best-known combinatorial opimization problem 
- **A nice book on the TSP:**  [In Pursuit of the Traveling Salesman](https://press.princeton.edu/books/paperback/9780691163529/in-pursuit-of-the-traveling-salesman)
 - die story of the TSP presented by one of its protagonists (William Cook)
- TSP website: https://www.math.uwaterloo.ca/tsp/index.html
- there are a lot of instances
    - in particular, there is a full library of instances, the so-called [TSPLib](http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/)
    - some of them are part of the git repository for the course material
    - [here](http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html) you find optimal objective values for many instances

**..and there is even a Python library dedicated to solving the TSP: [`python-tsp`](https://github.com/fillipe-gsm/python-tsp)** 

## The Python library `python-tsp`

see: https://github.com/fillipe-gsm/python-tsp

### offers:
- functions to read TSP instances in the tsplib-format
  
  
  

In [4]:
%%time
from python_tsp.distances import tsplib_distance_matrix

#tsplib_file = "./../problems/tsp/instances/a280.tsp" # optimal solution 2579 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)
tsplib_file = "./../problems/tsp/instances/brazil58.tsp" # optimal solution 25395 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)
#tsplib_file = "./../problems/tsp/instances/berlin52.tsp" # optimal solution  7542 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)

distance_matrix = tsplib_distance_matrix(tsplib_file)



CPU times: user 83.4 ms, sys: 16.3 ms, total: 99.8 ms
Wall time: 718 ms


- and heuristic as well as exact TSP algorithms
  - e.g. local search, simulated annealing and dynamic programming (exact: careful, may take very long)

In [5]:
%%time
from python_tsp.heuristics import solve_tsp_local_search, solve_tsp_simulated_annealing

#permutation, distance = solve_tsp_local_search(distance_matrix)

permutation, distance = solve_tsp_simulated_annealing(distance_matrix)
distance

CPU times: user 1 s, sys: 14.5 ms, total: 1.02 s
Wall time: 1.05 s


np.int64(26051)

## Representing a DP model in Python

- we can collect these three functions in a `NamedTuple` as follows:

In [6]:
class DP(NamedTuple):
    feasible_decisions : Callable
    transition_function : Callable
    cost_function : Callable
    direction : str # 'max' or 'min'

#### States
- we are free to define our state representation
- for later purposes, it will be useful if the state variable is immutable, therefore tuples or namedtuple are useful data structures for states


#### Decisions
- in most cases in this part, we will assume that decisions are integers, but note that this is not required as long as the transition function works
- however, for now, we assume that decisions only induce a change between stages -- we will relax that requirement later in the course


#### Instance data
- all functions named above take an instance as parameter. Instance data does not have to take a certain form, it just needs to "match" the (problem-specific) functions

## Generic helper functions to deal with maximization and minimization

In [7]:
@njit 
def better(value1, value2, direction):
    if direction == "min":
        return value1 < value2
    else:
        return value1 > value2


In [11]:
better(3,4,"min")

True

In [12]:
@njit
def best(collection, direction):
    if direction == "min":
        return min(collection)
    else:
        return max(collection)

In [13]:
example_values = [2,8,7,1,5]

print(best(example_values,"min"))
print(best(example_values,"max"))

1
8


In [14]:
from heapq import nsmallest
from heapq import nlargest

@njit 
def nbest(n,collection, direction):
    if direction == "min":
        return nsmallest(n,collection)
    else:
        return nlargest(n,collection)

In [15]:
print(nbest(2,example_values,"min"))
print(nbest(2,example_values,"max"))

[1, 2]
[8, 7]


## Example: A DP model for the Knapsack Problem

Given:
- a knapsack instance with $N$ items with weights $w_k$ and profits $p_k$ (zero-indexed) and capacity $W$ 

- state $x_k$: accumulated weight after adding the first $k-1$ items, $x_0 = 0$
- decision $u_k \in \{0, 1\}$ (0: do not add item $k$ to the knapsack; 1: add item $k$)
- $U_k(x_k) = \begin{cases} 
                \{0,1\} \quad \mathrm{if} \quad x_k + w_k \leq W \\
                \{0 \} \quad \mathrm{else}
\end{cases}$
- $f(x_k, u_k) = x_k + w_k u_k $
- $g(x_k, u_k) = p_k u_k$

We have a maximization-objective:

$$\max_{u_0,..,u_k,..u_{N-1}} \sum_{k=0}^{N-1} g_k(x_k,u_k)$$


## The Knapsack DP Model in Python

In [16]:
class  KPInstance(NamedTuple):
    values:np.array
    weights:np.array
    capacity:int
    N:int   

@njit
def feasible_decisions_kp(instance, k, acc_weight):    
    if acc_weight + instance.weights[k] <= instance.capacity: return [0,1]
    else: return [0]

@njit
def transition_function_kp(instance, k, acc_weight, put):
    return acc_weight + put*instance.weights[k]

@njit
def cost_function_kp(instance, k, acc_weight, put):
      return put*instance.values[k]

Putting all together, and stating that we have a maximization objective

In [17]:
dp_kp = DP(feasible_decisions_kp, transition_function_kp,  cost_function_kp, "max")

## An instance reader function for the Knapsack Problem

In [18]:

def read_kp_instance(filename, sorted=True):
    weights=[]
    values=[]
    with open(filename) as f: # open the file
        line = f.readline().split()  # split first row
        number_of_items = int(line[0]) # read number of items
        capacity = int(line[1]) # read capacity
        for i in range(number_of_items): # read rows for the items
            line = f.readline().split() # split row
            values.append(int(line[0])) # read value
            weights.append(int(line[1])) # read weight
            
    values = np.array(values)
    weights = np.array(weights)    
    
    
    if sorted:
        sorted_indexes = np.argsort(-1* values/weights)
    values = values[sorted_indexes]
    weights = weights[sorted_indexes]
     
        
    return KPInstance(values, weights, capacity, number_of_items)

In [19]:
filename = "./../problems/knapsack/instances/knapPI_1_5000_1000_1" # optimal value: 276457 
#filename = "./../problems/knapsack/instances/knapPI_1_100_1000_1"
kp_instance = read_kp_instance(filename)

## Example: A DP model for the TSP

Given:
- a TSP instance with a $N$ cities and distances $d_{i,j}$ between cities $i,j$
  - let us denote with $\mathcal{N} = \{1, \ldots N \}$ the set of cities 
  


- state $x_k$: sequence / ordered set of cities visited so far, $x_0 = i^0$ where $i^0$ is the first city
  - let us define $l(x_k)$ as the last element in the ordered set, that is, the "current" city

- decision $u_k \in \mathcal{N}$ city to visit next 
- $U_k(x_k) = \mathcal{N} \setminus x_k$
- $f(x_k, u_k) = x_k + u_k$  (here, with $+$ we mean to append $u_k$ to the sequence / ordered set $x_k$
- $g(x_k, u_k) = \begin{cases} 
                d_{l(x_k), u_k} \quad \mathrm{if} \quad k < N-1 \\
               d_{l(x_k), u_k} +  d_{u_k, i^0} \quad  \mathrm{if} \quad k = N-1
\end{cases}$

We have a minimization objective:

$$\min_{u_0,..,u_k,..u_{N-1}} \sum_{k=0}^{N-1} g_k(x_k,u_k)$$



## Example: DP model for the TSP in Python (slightly changed)

In [20]:
class TSPInstance(NamedTuple):
    distance_matrix : np.array
    N : int

@njit
def feasible_decisions_tsp(instance, k, sequence):
    if (k < instance.N-1):
        return np.array([i for i in range(instance.N+1) if i not in sequence])    
    else:
        return np.array([sequence[0]])  

@njit
def transition_function_tsp(instance, k, sequence, neighbor):
    if (k < instance.N-1):
        return sequence + [neighbor]
    else: 
        return sequence

@njit
def cost_function_tsp(instance, k, sequence, neighbor):    
    if k < instance.N-1:
        return instance.distance_matrix[sequence[k]][neighbor]
    else:
        return instance.distance_matrix[neighbor][sequence[0]]

dp_tsp = DP(feasible_decisions_tsp, transition_function_tsp,  cost_function_tsp, "min")

## An instance reader function for the TSP

In [21]:
def read_tsp_instance(filename):    
    distance_matrix = tsplib_distance_matrix(filename)
    return TSPInstance(distance_matrix, len(distance_matrix)-1)

In [23]:
tsplib_file = "./../problems/tsp/instances/brazil58.tsp" # optimal solution 25395 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)
#tsplib_file = "./../problems/tsp/instances/berlin52.tsp" # optimal solution  7542 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)

tsp_instance = read_tsp_instance(tsplib_file)

# Generic implementations of algorithms based on the generic DP model

## Generic greedy for DP models

- given a generic DP model, we can now start devising generic implementation of algorithms operating on DP models
- as an example, we can generically implement greedy as follows:

**Observe:**
- below, we avoid some loops by directly working on arrays of decisions


In [24]:
# @njit
def dp_greedy(dp, instance, state_start):
    
    state = state_start    
    solution_decisions = []    
    total_cost = 0
    
    for k in range(instance.N):          
        # get feasible decisions
        decisions = dp.feasible_decisions(instance, k, state)
        
        if decisions is None or len(decisions) == 0: continue            
        
        # create a tuple (cost, decision) for each decision
        costs_decisions =[(dp.cost_function(instance, k, state, d), d) for d in decisions]
        
        #get the best decision 
        best_cost, best_decision = best(costs_decisions, dp.direction) 
        
        #get target state
        state = dp.transition_function(instance, k, state, best_decision)

        total_cost += best_cost
        solution_decisions.append(best_decision)
        
    return total_cost, solution_decisions

## Exercise: Using a different state-transition model for the TSP

We can also model the TSP using the following state information:

- the start node (depot)
- the current node (location)
- the set of unvisited nodes

For the sake of simplicity, let us from now on assume that node 0 is the depot, that is, we do not have to store the depot in the state.

Create a DP-Model for this.

Hint: use either a tuple or a namedtuple




In [25]:
# we assume that the start node is always zero (for the TSP it doesn't actually matter)
class TSPState(NamedTuple):
    idx: int
    unvisited: set

def feasible_decisions_tsp_new(instance: TSPInstance, k: int, tsp_state: TSPState):
    if (k < instance.N-1):
        return list(tsp_state.unvisited)
    else:
        return [0]
    
def transition_function_tsp_new(instance: TSPInstance, k:int, tsp_state: TSPState, neighbor_idx: int):
    if k < instance.N-1:
        return TSPState(neighbor_idx, tsp_state.unvisited - set([neighbor_idx]))
    else:
        assert not tsp_state.unvisited, "Not all nodes have been visited"
        return TSPState(0, tsp_state.unvisited)
    
def cost_function_tsp_new(instance: TSPInstance, k: int, tsp_state: TSPState, neighbor_idx: int):
    return instance.distance_matrix[tsp_state.idx][neighbor_idx]

dp_tsp_new = DP(
    feasible_decisions=feasible_decisions_tsp_new, 
    transition_function=transition_function_tsp_new, 
    cost_function=cost_function_tsp_new, 
    direction="min"
)

In [26]:
start_state = TSPState(0, set(range(1, tsp_instance.N)))
dp_greedy(dp=dp_tsp_new, instance=tsp_instance, state_start=start_state)[0]

30799

## Exercise: TSP with Time Windows


Let us now consider the TSP-TW in which
- each customer $i$ (witha a node index > 0) is associated with a time window $[a_i, b_i]$ within which they have to be visited
- if we arrive at $i$ before $a_i$, we have to wait at $a_i$
- if we arrive later than $b_i$ at a node $i$, the solution is infeasible




In [27]:
class TSPTW_Instance(NamedTuple):
    distance_matrix : np.array
    a : np.array
    b : np.array
    N : int


def read_tsptw_instance(filename):
    with open(filename) as f:
        values = f.read().split()

    position = 0
    n = int(values[position])
    position += 1
    nodes = list(range(n))
    edges = {}
    for i in range(n):
        for j in range(n):
            if i != j:
                try:
                    edges[i, j] = int(values[position])
                except ValueError:
                    edges[i, j] = float(values[position])
            position += 1
    a = {}
    b = {}
    for i in range(n):
        try:
            a[i] = int(values[position])
        except ValueError:
            a[i] = float(values[position])
        position += 1
        try:
            b[i] = int(values[position])
        except ValueError:
            b[i] = float(values[position])
        position += 1
        
    distance_matrix = [
        [edges[i, j] if (i, j) in edges else 0 for j in nodes] for i in nodes
    ]

    return TSPTW_Instance(distance_matrix, list(a.values()), list(b.values()), n)

In [28]:
# Number of locations
n = 4
# Ready time
a = [0, 5, 0, 8]
# Due time
b = [100, 16, 10, 14]
# Travel time
c = [
    [0, 3, 4, 5],
    [3, 0, 5, 4],
    [4, 5, 0, 3],
    [5, 4, 3, 0],
]

tsptw_instance_tiny = TSPTW_Instance(c,a,b,n)

filename = "./../problems/tsptw/Dumas/n80w60.001.txt"
tsptw_instance = read_tsptw_instance(filename)

## Create a DP Model for the TSP-TW

- use our new model as the basis for a DP model for the TSP-TW
- how should we modify the state variable?
- how do we extend the other parts of the DP model?






In [29]:
class TSPTWState(NamedTuple):
    idx: int
    unvisited: set
    curr_time: float

def feasible_decisions_tsptw(instance: TSPTW_Instance, k: int, tsptw_state: TSPTWState):
    if k < instance.N-1:
        feasible_next_nodes = []
        for node in tsptw_state.unvisited:
            if tsptw_state.curr_time + instance.distance_matrix[tsptw_state.idx][node] <= instance.b[node]:
                feasible_next_nodes.append(node)
        return feasible_next_nodes
    else:
        return [0]
    
def transition_function_tsptw(instance: TSPTW_Instance, k:int, tsptw_state: TSPTWState, neighbor_idx: int):
    if k < instance.N-1:
        travel_time = instance.distance_matrix[tsptw_state.idx][neighbor_idx]
        return TSPTWState(
            idx=neighbor_idx, 
            unvisited=tsptw_state.unvisited - {neighbor_idx}, 
            curr_time=max(tsptw_state.curr_time + travel_time, instance.a[neighbor_idx])
        )
    else:
        assert not tsptw_state.unvisited, f"Some nodes have not been visited: {list(tsptw_state.unvisited)}"
        return TSPTWState(idx=0, unvisited=tsptw_state.unvisited, curr_time=tsptw_state.curr_time + instance.distance_matrix[tsptw_state.idx][0])
    
dp_tsptw = DP(
    feasible_decisions=feasible_decisions_tsptw, 
    transition_function=transition_function_tsptw, 
    cost_function=cost_function_tsp_new, 
    direction="min"
)

In [30]:
# tiny instance
start_state_tiny = TSPTWState(idx=0, unvisited=set(range(1, tsptw_instance_tiny.N)), curr_time=0)
dp_greedy(dp=dp_tsptw, instance=tsptw_instance_tiny, state_start=start_state_tiny)

AssertionError: Some nodes have not been visited: [2]

In [31]:
# other instance
start_state = TSPTWState(idx=0, unvisited=set(range(1, tsptw_instance.N)), curr_time=0)
dp_greedy(dp=dp_tsptw, instance=tsptw_instance, state_start=start_state)[0]

AssertionError: Some nodes have not been visited: [1, 3, 4, 7, 9, 10, 11, 12, 15, 16, 17, 18, 20, 21, 22, 24, 25, 26, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79]

## Domain-Independent Dynamic Programming (DIDP)

### Domain-Independent Dynamic Programming (DIDP)

- in our simple implementation, we aimed at a generic DP modeling framework that can be used in a model-and-solve fashion
  - generic alorithms that can solve COs formulated as DP models
  
- this idea is also followed (in a much more sophisticated way) by the framework **Domain-Independent Dynamic Programming (DIDP)** recently introduced by Ryo Kuroiwa and J. C Beck: https://arxiv.org/abs/2401.13883

- the software is implemented in Rust and has a Python interface called **[DIDPPy](https://didppy.readthedocs.io/en/stable/index.html)**



### Example: A DIDP model for the 0/1 Knapsack Problem
(we our instance data to feed the model)

In [32]:
filename = "./../problems/knapsack/instances/knapPI_1_5000_1000_1" # optimal value: 276457 
filename = "./../problems/knapsack/instances/knapPI_1_2000_1000_1" # optimal value: 110625

kp_instance = read_kp_instance(filename)

In [37]:
import didppy as dp

#the model
model = dp.Model(maximize=True, float_cost=False)

## adding the data
w = model.add_int_table(kp_instance.weights)
p = model.add_int_table(kp_instance.values)

# adding the "objects"
item = model.add_object_type(number=kp_instance.N)

## adding the state variables
i = model.add_element_var(object_type=item, target=0) # index / stage variable

r = model.add_int_var(target=kp_instance.capacity) # "remaining" capacity, and equivalent model


###### the transitions are explicitly introduced
pack = dp.Transition(
    name="pack",
    cost=p[i] + dp.IntExpr.state_cost(),
    effects=[(r, r - w[i]), (i, i + 1)],
    preconditions=[i < kp_instance.N, r >= w[i]],)

model.add_transition(pack)

ignore = dp.Transition(
    name="ignore",
    cost=dp.IntExpr.state_cost(),
    effects=[(i, i + 1)],
    preconditions=[i < kp_instance.N], )

model.add_transition(ignore)

model.add_base_case([i == kp_instance.N]) # end

### Example: A DIDP model for the 0/1 Knapsack Problem
(we our instance data to feed the model)

In [38]:
%%time
solver = dp.ForwardRecursion(model)
#solver = dp.CABS(model, time_limit=60)
solution = solver.search()
solution.cost

Solver: ForwardRecursion from DIDPPy v0.8.0
CPU times: user 41.6 s, sys: 30.1 s, total: 1min 11s
Wall time: 1min 18s


110625

### The model in detail

##### Objects and element variables

- objects are sort of the relevant entities in the model (items, cities, customers, etc.)
- in many cases they give rise to the number of states

In [39]:
item = model.add_object_type(number=kp_instance.N)

- objects are indexed by element variables
- these elements are the ids of the objects (typically integers), and they are usually part of the state variable
- in many models, they have more or less the same function as the "stage index" $k$ in our models above
- `target` is the inital value (the name target comes from the fact that DP is seen as a backward alorithm here)

In [40]:
i = model.add_element_var(object_type=item, target=0) # index / stage variable


### The model in detail

##### State variables
- there are several other variable types (int, continuous, set-valued) that can serve as state attributes / members of the compound state variable
- in the knapsack model, we have:

In [41]:
r = model.add_int_var(target=kp_instance.capacity) # "remaining" capacity,


- thus, here, we have both i and r as parts of the state variable

## State Transitions

State transition are handled completely differently than in our formalism above:
- there is one `Transition` object for each possible decision (type) which is added to the model
- the set of feasible decisions / the question if a transition can be applied is handled by so-called `preconditions`
- the `Transition` object handles both the change in cost and the state transition which is called `effect` in DIDP

In [42]:
pack = dp.Transition(
    name="pack",
    cost=p[i] + dp.IntExpr.state_cost(),
    effects=[(r, r - w[i]), (i, i + 1)],
    preconditions=[i < kp_instance.N, r >= w[i]],)

model.add_transition(pack)

ignore = dp.Transition(
    name="ignore",
    cost=dp.IntExpr.state_cost(),
    effects=[(i, i + 1)],
    preconditions=[i < kp_instance.N], )

model.add_transition(ignore)

### Base Case and Solving the Problem

Finally, one adds the so-called base case(s), that is, final states:

In [43]:
model.add_base_case([i == kp_instance.N]) # end

### Once again the full model


In [44]:

filename = "./../problems/knapsack/instances/knapPI_1_5000_1000_1" # optimal value: 276457 
filename = "./../problems/knapsack/instances/knapPI_1_2000_1000_1" # optimal value: 110625
filename = "./../problems/knapsack/instances/knapPI_1_100_1000_1"

kp_instance = read_kp_instance(filename)
#the model
model = dp.Model(maximize=True, float_cost=False)

## adding the data
w = model.add_int_table(kp_instance.weights)
p = model.add_int_table(kp_instance.values)

# adding the "objects"
item = model.add_object_type(number=kp_instance.N)

## adding the state variables
i = model.add_element_var(object_type=item, target=0) # index / stage variable

r = model.add_int_resource_var(target=kp_instance.capacity, less_is_better=False) # "remaining" capacity, and equivalent model 


###### the transitions are explicitly introduced
pack = dp.Transition(
    name="pack",
    cost=p[i] + dp.IntExpr.state_cost(),
    effects=[(r, r - w[i]), (i, i + 1)],
    preconditions=[i < kp_instance.N, r >= w[i]],)

model.add_transition(pack)

ignore = dp.Transition(
    name="ignore",
    cost=dp.IntExpr.state_cost(),
    effects=[(i, i + 1)],
    preconditions=[i < kp_instance.N], )

model.add_transition(ignore)

model.add_base_case([i == kp_instance.N]) # end

### Solving the model

DIDP has several solvers implemented, e.g.:
- a forward DP approach akin to the DP by reaching algorithm
- Complete Anytime Beam Search (CABS)
- a variant of Anytime Column Search (ACPS)
- an AI planning algorithm (CAASDy)

.. for a nice presentation dealing with some of these search algorithms, see these [slides](https://drive.google.com/file/d/1qaE4jlz8XkKCQL7QdZ-VrTQi21cYvGco/view) from Christine Solnon (INSA Lyon)

In [45]:
%%time
solver = dp.ForwardRecursion(model)
#solver = dp.CABS(model, time_limit=60)
solution = solver.search()
solution.cost

Solver: ForwardRecursion from DIDPPy v0.8.0
CPU times: user 88.6 ms, sys: 43.9 ms, total: 132 ms
Wall time: 175 ms


9147

## Dominance

Often, we can detect states dominated by others:

State $S_1$ dominates $S_2$

- their set of partial completions (solutions starting from that state) of $S_1$ is a superset of that of $S_2$: $Sol(S_1) \supseteq Sol(S_2)$
- the objective value of $S_1$ is at least as good as that of $S_2$


For the knapsack problem, a state $S_1$  dominates $S_2$ other if both states are in the same layer and:
- its accumulated weight is smaller, that is,  $w_{S_1} \leq w_{S_2}$
- and its objective is at least as good, that is, $f_{S_1} \geq f_{S_2}$

--> Dominated states can be ignored during the search!!



## Dominance: Illustration


<img src="./img/dp/dominance_1.png" width=80%>



## Dominance: Illustration


<img src="./img/dp/dominance_2.png" width=80%>



## Dominance: Illustration


<img src="./img/dp/dominance_3.png" width=80%>



## Dominance Handling in DIDP

DIDP provides a simple way for handling and exploiting (simple) dominance relations:

- instead of using standard state variables, one can use so-called **resource variables**
- for a resource variable, one indicates wether less is better or not
- then, if for two states, if all state attributes (e.g. the element variables) but the values of the resource variables are identical, only the state with the better value in the resource variables is kept.




For the KP, we can simply replace the (standard) state variable definition:


In [46]:
r = model.add_int_var(target=kp_instance.capacity)

with a resource variable as follows

In [47]:
model.add_int_resource_var(target=kp_instance.capacity, less_is_better=False)

## Dominance for the DIDP KP Model: Full model



In [48]:
model = dp.Model(maximize=True, float_cost=False)
w = model.add_int_table(kp_instance.weights)
p = model.add_int_table(kp_instance.values)

item = model.add_object_type(number=kp_instance.N)
r = model.add_int_resource_var(target=kp_instance.capacity, less_is_better=False)
i = model.add_element_var(object_type=item, target=0)
pack = dp.Transition(
    name="pack",
    cost=p[i] + dp.IntExpr.state_cost(),
    effects=[(r, r - w[i]), (i, i + 1)],
    preconditions=[i < kp_instance.N, r >= w[i]],
)
model.add_transition(pack)

ignore = dp.Transition(
    name="ignore",
    cost=dp.IntExpr.state_cost(),
    effects=[(i, i + 1)],
    preconditions=[i < kp_instance.N],
)

model.add_transition(ignore)

model.add_base_case([i == kp_instance.N])

In [49]:
%%time
#solver = dp.ForwardRecursion(model)
solver = dp.CABS(model)
solution = solver.search()


solution.cost

Solver: CABS from DIDPPy v0.8.0
CPU times: user 14.5 ms, sys: 1.46 ms, total: 15.9 ms
Wall time: 19.4 ms
Searched with beam size: 1, expanded: 100, elapsed time: 0.001682125
New primal bound: 8817, expanded: 100, elapsed time: 0.001871417
Searched with beam size: 2, expanded: 299, elapsed time: 0.002505126
New primal bound: 8929, expanded: 299, elapsed time: 0.002585709
Searched with beam size: 4, expanded: 692, elapsed time: 0.002976751
Searched with beam size: 8, expanded: 1469, elapsed time: 0.003824709
New primal bound: 9147, expanded: 1469, elapsed time: 0.003876042
Searched with beam size: 16, expanded: 3000, elapsed time: 0.005237834
Searched with beam size: 32, expanded: 6004, elapsed time: 0.010926876
Searched with beam size: 64, expanded: 11303, elapsed time: 0.017498418


9147

## DIDP Model for the TSPTW (Part 1)
- we first use a 20-city instance, then a 60-city instance

In [50]:
filename = "./../problems/tsptw/Dumas/n20w60.001.txt" #(optimal solution 400)
#filename = "./../problems/tsptw/Dumas/n80w60.001.txt" #(optimal solution 554)
tsptw_instance = read_tsptw_instance(filename)

In [51]:
model = dp.Model(maximize=False, float_cost=False)

### the data
ready_time = model.add_int_table(tsptw_instance.a)
due_time = model.add_int_table(tsptw_instance.b)
travel_time = model.add_int_table(tsptw_instance.distance_matrix)

# customers (nodes) are the main object in this problem
customer = model.add_object_type(number=tsptw_instance.N)

## now the state variable components:
# location is an element variable (index for the set of customers / nodes)
idx = model.add_element_var(object_type=customer, target=0)

# unvisited (nodes) is a so-called set variable!
unvisited = model.add_set_var(object_type=customer, target=list(range(1, tsptw_instance.N)))

# state component for time
curr_time = model.add_int_var(target=0)

## DIDP Model for the TSPTW (Part II)

In [52]:
# now, we add one transition per location
for j in range(1, tsptw_instance.N):
    visit = dp.Transition(
        name="visit {}".format(j),
        cost=travel_time[idx, j] + dp.IntExpr.state_cost(),
        preconditions=[unvisited.contains(j),
                      curr_time + travel_time[idx, j] <= due_time[j]],
        effects=[
            (unvisited, unvisited.remove(j)),
            (idx, j),
            (curr_time, dp.max(curr_time + travel_time[idx, j], ready_time[j])),
        ],
    )
    model.add_transition(visit)

# and finally, one for returning to the depot
return_to_depot = dp.Transition(
    name="return",
    cost=travel_time[idx, 0] + dp.IntExpr.state_cost(),
    effects=[
        (idx, 0),
        (curr_time, curr_time + travel_time[idx, 0]),
    ],
    preconditions=[unvisited.is_empty(), idx != 0],
)

model.add_transition(return_to_depot)

## and the  base case
model.add_base_case([unvisited.is_empty(), idx == 0])

### Solving the Model
- we impose a 60-second time limit 
- the optimal solution for the instance `n20w60.001.txt` is   335
- the optimal solution for the instance `n80w60.001.txt` is   554 


In [53]:
%%time
solver = dp.CABS(model, time_limit=60)
solution = solver.search()



print("Cost: {}".format(solution.cost))

Solver: CABS from DIDPPy v0.8.0
Cost: 335
CPU times: user 28.2 s, sys: 336 ms, total: 28.6 s
Wall time: 28.5 s
Searched with beam size: 1, expanded: 6, elapsed time: 0.001068458
Searched with beam size: 2, expanded: 23, elapsed time: 0.001213333
Searched with beam size: 4, expanded: 84, elapsed time: 0.001714208
Searched with beam size: 8, expanded: 210, elapsed time: 0.002741916
Searched with beam size: 16, expanded: 504, elapsed time: 0.004992708
New primal bound: 344, expanded: 504, elapsed time: 0.004995
Searched with beam size: 32, expanded: 1070, elapsed time: 0.010574916
Searched with beam size: 64, expanded: 2164, elapsed time: 0.017551541
New primal bound: 335, expanded: 2164, elapsed time: 0.017578458
Searched with beam size: 128, expanded: 4312, elapsed time: 0.029377791
Searched with beam size: 256, expanded: 8492, elapsed time: 0.051703541
Searched with beam size: 512, expanded: 16453, elapsed time: 0.094166125
Searched with beam size: 1024, expanded: 31818, elapsed time: 

## A Dominance Relation for the TSP-TW

For the TSP-TW, there is a very simple dominance relation: 
- if all other state components are identical, the state with a smaller time is better
- thus, we turn the state variable time into a resource variable:

In [ ]:
curr_time = model.add_int_resource_var(target=0, less_is_better=True)

## The TSP-TW Model including the Dominance Relation

In [ ]:
model = dp.Model(maximize=False, float_cost=False)

### the data
ready_time = model.add_int_table(tsptw_instance.a)
due_time = model.add_int_table(tsptw_instance.b)
travel_time = model.add_int_table(tsptw_instance.distance_matrix)

# customers (nodes) are the main object in this problem
customer = model.add_object_type(number=tsptw_instance.N)

## now the state variable components:
# location is an element variable (index for the set of customers / nodes)
idx = model.add_element_var(object_type=customer, target=0)

# unvisited (nodes) is a so-called set variable!
unvisited = model.add_set_var(object_type=customer, target=list(range(1, tsptw_instance.N)))

# NEW: state component for time becomes a resource variable
curr_time = model.add_int_resource_var(target=0, less_is_better=True)

# now, we add one transition per location
for j in range(1, tsptw_instance.N):
    visit = dp.Transition(
        name="visit {}".format(j),
        cost=travel_time[idx, j] + dp.IntExpr.state_cost(),
        preconditions=[unvisited.contains(j), 
                       curr_time + travel_time[idx, j] <= due_time[j]],
        effects=[
            (unvisited, unvisited.remove(j)),
            (idx, j),
            (curr_time, dp.max(curr_time + travel_time[idx, j], ready_time[j])),
        ],
    )
    model.add_transition(visit)

# and finally, one for returning to the depot
return_to_depot = dp.Transition(
    name="return",
    cost=travel_time[idx, 0] + dp.IntExpr.state_cost(),
    effects=[
        (idx, 0),
        (curr_time, curr_time + travel_time[idx, 0]),
    ],
    preconditions=[unvisited.is_empty(), idx != 0],
)

model.add_transition(return_to_depot)

## and the  base case
model.add_base_case([unvisited.is_empty(), idx == 0])

## The TSP-TW Model including the Dominance Relation: Let us solve it

In [ ]:
%%time
solver = dp.CABS(model, time_limit=30)
solution = solver.search()



print("Cost: {}".format(solution.cost))

## State Constraints

**Observation:**

- a state is a deadend if there is a single unvisited neighbor cannot be reached before its deadline
- this is discovered sooner or later, but that may take considerable time, so better check each state for that condition
- in DIDP, we can impose so-called *state constraints*

In [ ]:
## add a state constraint for each customer:
## each customer that has not been visited must be reachable from the current location before its due time
for j in range(1, tsptw_instance.N):
    model.add_state_constr(
        ~unvisited.contains(j) | (curr_time + travel_time[idx, j] <= due_time[j])
    )

## The full model with state constraints

In [ ]:
filename = "./../problems/tsptw/Dumas/n20w60.001.txt" #(optimal solution 400)
#filename = "./../problems/tsptw/Dumas/n80w60.001.txt" #(optimal solution 554)
tsptw_instance = read_tsptw_instance(filename)

model = dp.Model(maximize=False, float_cost=False)

### the data
ready_time = model.add_int_table(tsptw_instance.a)
due_time = model.add_int_table(tsptw_instance.b)
travel_time = model.add_int_table(tsptw_instance.distance_matrix)

# customers (nodes) are the main object in this problem
customer = model.add_object_type(number=tsptw_instance.N)

## now the state variable components:
# location is an element variable (index for the set of customers / nodes)
idx = model.add_element_var(object_type=customer, target=0)

# unvisited (nodes) is a so-called set variable!
unvisited = model.add_set_var(object_type=customer, target=list(range(1, tsptw_instance.N)))

# NEW: state component for time becomes a resource variable
curr_time = model.add_int_resource_var(target=0, less_is_better=True)

# now, we add one transition per location
for j in range(1, tsptw_instance.N):
    visit = dp.Transition(
        name="visit {}".format(j),
        cost=travel_time[idx, j] + dp.IntExpr.state_cost(),
        preconditions=[unvisited.contains(j), 
                       curr_time + travel_time[idx, j] <= due_time[j]],
        effects=[
            (unvisited, unvisited.remove(j)),
            (idx, j),
            (curr_time, dp.max(curr_time + travel_time[idx, j], ready_time[j])),
        ],
    )
    model.add_transition(visit)

# and finally, one for returning to the depot
return_to_depot = dp.Transition(
    name="return",
    cost=travel_time[idx, 0] + dp.IntExpr.state_cost(),
    effects=[
        (idx, 0),
        (curr_time, curr_time + travel_time[idx, 0]),
    ],
    preconditions=[unvisited.is_empty(), idx != 0],
)

model.add_transition(return_to_depot)

## the state constraint
for j in range(1, tsptw_instance.N):
    model.add_state_constr(
        ~unvisited.contains(j) | (curr_time + travel_time[idx, j] <= due_time[j])
    )

## and the  base case
model.add_base_case([unvisited.is_empty(), idx == 0])

## Conclusions: DIDP
- a great free framework for exactly solving CO problems that have a good DP formulation
- the efficiency highly depends on the quality of the formulation
    - just as in MIP, for the same problem there are often formulations that are easier to solve than others
- one aspect we did not mention at all so far are **state-based dual bounds**
  - if we can provide a dual bound for the objective of the tail subproblem of a given state, many of the algorithms can perfrom state pruning
   - what could be state-based dual bounds for the KP and the TSP-TW?